In [17]:
# Install all required libraries
!pip install transformers datasets pandas peft accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import pandas as pd
import logging
import random

In [20]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Enhanced Emoji Math Dataset with more variety
data = [
    {"problem": "🍎 + 🍎 + 🍎 = 12", "solution": "🍎 = 4"},
    {"problem": "🚗 + 🚗 + 🚗 + 🚗 = 20", "solution": "🚗 = 5"},
    {"problem": "🎸 ÷ 4 = 5", "solution": "🎸 = 20"},
    {"problem": "⚽ ÷ 3 = 6", "solution": "⚽ = 18"},
    {"problem": "🐶 + 🐶 = 10", "solution": "🐶 = 5"},
    {"problem": "🍕 + 🍕 + 🍕 = 18", "solution": "🍕 = 6"},
    {"problem": "🐼 + 🐼 + 🐼 + 🐼 = 24", "solution": "🐼 = 6"},
    {"problem": "🍓 + 🍓 + 🍓 = 18", "solution": "🍓 = 6"},
    {"problem": "🚙 + 🚙 + 🚙 = 27", "solution": "🚙 = 9"},
    {"problem": "🐨 + 🐨 + 🐨 = 18", "solution": "🐨 = 6"},
    {"problem": "🍒 + 🍒 + 🍒 + 🍒 = 20", "solution": "🍒 = 5"},
    {"problem": "🚁 + 🚁 = 16", "solution": "🚁 = 8"},
    {"problem": "🐝 + 🐝 + 🐝 = 21", "solution": "🐝 = 7"},
    {"problem": "🍌 + 🍌 + 🍌 = 18", "solution": "🍌 = 6"},
    {"problem": "🚤 + 🚤 = 14", "solution": "🚤 = 7"},
    {"problem": "🐒 + 🐒 + 🐒 + 🐒 = 24", "solution": "🐒 = 6"},
    {"problem": "🍎 × 🍎 = 16", "solution": "🍎 = 4"},
    {"problem": "🚗 - 🚗 = 0", "solution": "🚗 = any number"},
    {"problem": "🐶 × 2 = 10", "solution": "🐶 = 5"},
    {"problem": "🍕 ÷ 2 = 3", "solution": "🍕 = 6"},
    {"problem": "🎈 × 🎈 = 25", "solution": "🎈 = 5"},
    {"problem": "🍊 - 🍊 = 0", "solution": "🍊 = any number"},
    {"problem": "🐢 - 🐢 - 🐢 = -6", "solution": "🐢 = 2"},
    {"problem": "🚴‍♂️ - 🚴‍♂️ - 🚴‍♂️ - 🚴‍♂️ = -12", "solution": "🚴‍♂️ = 3"},
    {"problem": "🐘 + 🐘 + 🐘 + 🐘 = 28", "solution": "🐘 = 7"},
    {"problem": "🍇 + 🍇 + 🍇 = 15", "solution": "🍇 = 5"},
    {"problem": "🐦 + 🐦 + 🐦 + 🐦 = 24", "solution": "🐦 = 6"},
    {"problem": "🐸 ÷ 3 = 7", "solution": "🐸 = 21"},
    {"problem": "🎃 ÷ 2 = 9", "solution": "🎃 = 18"},
    {"problem": "🎈 + 🎈 + 🎈 + 🎈 + 🎈 = 25", "solution": "🎈 = 5"},
    {"problem": "🍦 + 🍦 = 8", "solution": "🍦 = 4"},
    {"problem": "🍔 + 🍔 + 🍔 = 18", "solution": "🍔 = 6"},
    {"problem": "🚕 + 🚕 + 🚕 = 27", "solution": "🚕 = 9"},
    {"problem": "🐠 + 🐠 + 🐠 + 🐠 + 🐠 = 25", "solution": "🐠 = 5"},
    {"problem": "🍰 + 🍰 + 🍰 = 21", "solution": "🍰 = 7"},
    {"problem": "🚚 + 🚚 = 18", "solution": "🚚 = 9"},
    {"problem": "🏀 ÷ 5 = 4", "solution": "🏀 = 20"},
    {"problem": "🍔 ÷ 6 = 3", "solution": "🍔 = 18"},
    {"problem": "🦁 - 🦁 - 🦁 = -9", "solution": "🦁 = 3"},
    {"problem": "🐇 - 🐇 - 🐇 - 🐇 - 🐇 = -10", "solution": "🐇 = 2"},
    {"problem": "🍍 - 🍍 = 0", "solution": "🍍 = any number"},
    {"problem": "🐯 - 2 × 🐯 = -5", "solution": "🐯 = 5"},
    {"problem": "🐎 - 3 × 🐎 = -6", "solution": "🐎 = 3"},
    {"problem": "🍉 ÷ 2 = 4", "solution": "🍉 = 8"},
    {"problem": "🦄 ÷ 5 = 3", "solution": "🦄 = 15"},
    {"problem": "🚢 ÷ 4 = 6", "solution": "🚢 = 24"},
    {"problem": "🐱 + 🐱 + 🐱 = 15", "solution": "🐱 = 5"},
    {"problem": "🚲 + 🚲 = 14", "solution": "🚲 = 7"},
    {"problem": "🍩 + 🍩 + 🍩 + 🍩 = 24", "solution": "🍩 = 6"},
    {"problem": "🐧 + 🐧 + 🐧 = 21", "solution": "🐧 = 7"},
    {"problem": "🍫 + 🍫 + 🍫 + 🍫 + 🍫 = 30", "solution": "🍫 = 6"},
    {"problem": "🚀 + 🚀 = 16", "solution": "🚀 = 8"},
    {"problem": "🏎️ ÷ 2 = 10", "solution": "🏎️ = 20"}
]


# Convert to DataFrame and save
df = pd.DataFrame(data)
df.to_csv("emoji_math_dataset.csv", index=False)
print("Dataset saved as 'emoji_math_dataset.csv'")

Dataset saved as 'emoji_math_dataset.csv'


In [25]:
# Prepare dataset with combined problem+solution
dataset = Dataset.from_pandas(df)
dataset = dataset.map(lambda x: {"text": f"{x['problem']} → {x['solution']}"})

# Load model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Enable 8-bit quantization
    device_map="auto"
)

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=32,
    target_modules=["c_attn"],  # Target attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Tokenization function
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=32,
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"]
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./emoji_math_solver",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    learning_rate=3e-4,  # Slightly higher for LoRA
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    report_to="none",
    fp16=True  # Enable mixed precision training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train and save
trainer.train()
model.save_pretrained("emoji_math_solver")
tokenizer.save_pretrained("emoji_math_solver")

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quanti

Step,Training Loss
100,1.683100


('emoji_math_solver/tokenizer_config.json',
 'emoji_math_solver/special_tokens_map.json',
 'emoji_math_solver/vocab.json',
 'emoji_math_solver/merges.txt',
 'emoji_math_solver/added_tokens.json',
 'emoji_math_solver/tokenizer.json')

In [28]:
# Testing
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

test_problems = [
    "🚗 + 🚗 + 🚗 + 🚗 = 20 → 🚗 =",
    "🍕 + 🍕 + 🍕 = 18 → 🍕 =",
    "🐶 + 🐶 + 🐶 = 15 → 🐶 =",
    "🍎 × 🍎 = 16 → 🍎 =",
    "🍌 ÷ 2 = 3 → 🍌 ="
]

print("\nTesting Emoji Math Solver:")
for problem in test_problems:
    output = generator(
        problem,
        max_length=32,
        num_return_sequences=1,
        temperature=0.7,
        top_k=40,
        top_p=0.9,
        truncation=True
    )
    print(f"Problem: {problem}")
    print(f"Solution: {output[0]['generated_text']}")
    print("---")

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa


Testing Emoji Math Solver:
Problem: 🚗 + 🚗 + 🚗 + 🚗 = 20 → 🚗 =
Solution: 🚗 + 🚗 + 🚗 + 🚗 = 20 → 🚗 = 5
---
Problem: 🍕 + 🍕 + 🍕 = 18 → 🍕 =
Solution: 🍕 + 🍕 + 🍕 = 18 → 🍕 = 6
---
Problem: 🐶 + 🐶 + 🐶 = 15 → 🐶 =
Solution: 🐶 + 🐶 + 🐶 = 15 → 🐶 = 6
---
Problem: 🍎 × 🍎 = 16 → 🍎 =
Solution: 🍎 × 🍎 = 16 → 🍎 = 1
---
Problem: 🍌 ÷ 2 = 3 → 🍌 =
Solution: 🍌 ÷ 2 = 3 → 🍌 = 🍌 = 6
---
